Test action space sample

In [2]:
from stable_baselines3 import PPO
import edge_relocation as er
import datetime



Test NON masks

In [2]:


max_trajectory_length = 25
min_trajectory_length = 25
initial_load = 'variable_load' # low (10-40%), medium(40-60%)), high(60-80%), random (10-80%), variable_load ( different initial load for each episode)

# create environment

erEnv = er.EdgeRelEnv("topoconfig.json", min_trajectory_length, max_trajectory_length, initial_load)

log_dir = "../logs/PPO/non_masked" + datetime.datetime.now().strftime("TEST")

model = PPO("MultiInputPolicy", erEnv, verbose=0, tensorboard_log = log_dir, n_steps=64)
model.learn(total_timesteps=50000)
model.save("model-non-masked-25trajectory_newReward_variableload"+datetime.datetime.now().strftime("Test_NONMask_250trajectory_newReward%Y%m%d-%H%M%S"))



In [3]:

max_trajectory_length = 25
min_trajectory_length = 25
initial_load = 'variable_load' # low (10-40%), medium(40-60%)), high(60-80%), random (10-80%), variable_load ( different initial load for each episode)

# create environment

erEnv = er.EdgeRelEnv("topoconfig.json", min_trajectory_length, max_trajectory_length, initial_load)
log_dir = "../logs/PPO/non_masked" + datetime.datetime.now().strftime("TEST")
model = PPO("MultiInputPolicy", erEnv, verbose=0, tensorboard_log = log_dir, n_steps=64)
model.learn(total_timesteps=50000)

obs = erEnv.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = erEnv.step(action)
    print(action)

1
18
22
17
20
9
18
17
7
14
16
17
19
4
4
17
17
17
4
16
1
17
1
17
17
4
1
7
17
4
17
17
17
21
4
16
17
8
7
4
18
4
4
18
18
17
4
9
17
6
20
4
2
17
17
4
4
16
4
17
9
4
19
14
18
18
17
18
19
4
8
17
4
17
4
4
11
17
19
4
17
4
17
17
17
14
4
17
17
18
17
17
17
15
17
16
4
4
17
14
19
1
17
19
17
17
17
8
18
17
4
4
4
17
4
4
17
16
17
19
4
8
16
10
18
4
5
17
17
16
18
8
18
19
17
8
4
17
7
14
17
20
18
14
17
17
17
4
4
7
17
4
16
17
17
15
16
17
4
4
17
10
4
3
4
9
17
4
17
17
14
8
4
4
2
16
8
4
17
5
4
17
14
18
17
19
17
17
17
7
4
18
18
17
4
18
15
17
20
4
17
16
4
17
4
17
17
17
18
16
4
4
17
17
17
17
17
17
4
4
4
14
17
4
1
1
7
1
18
17
1
17
18
16
17
16
17
17
21
15
19
18
4
11
8
4
4
4
17
4
19
17
17
14
22
4
17
16
4
22
17
4
1
8
17
17
3
17
9
6
19
17
17
17
17
17
2
8
17
4
8
17
17
17
1
17
17
14
17
17
3
11
20
17
4
14
4
17
17
14
10
19
17
16
4
16
17
3
16
4
4
17
17
19
17
1
16
17
16
18
7
16
16
14
4
17
13
16
17
17
16
13
4
16
17
17
11
19
8
17
16
17
17
19
1
15
13
12
18
1
4
18
17
4
17
7
4
17
16
17
18
13
17
17
4
17
4
17
4
4
14
4
17
3
17
16
11
1

KeyboardInterrupt: 

Test Maskable PPO

In [2]:
# import edge_relocation as er
#
# import datetime
#
#
# # create environment
# erEnv = er.EdgeRelEnv("topoconfig.json")
# # obs = erEnv.reset()
# log_dir = "../logs/PPO/" + datetime.datetime.now().strftime("Test_Mask_high_load_%Y%m%d-%H%M%S")
# model = MaskablePPO("MultiInputPolicy", erEnv, verbose=0, tensorboard_log = log_dir, n_steps=64)
# model.learn(total_timesteps=5000)
# print("episodes number: ", erEnv.episodes_counter)
#
#
# #Seems that for today, MaskablePPO is checking about the type of Space, and it accepts only spaces of gym, not yet a spaces of gymnasium. Further checks required

ValueError: Environment does not support action masking. Consider using ActionMasker wrapper

5566545+454+5412384049++8049+Test manually single episode

In [17]:
erEnv = er.EdgeRelEnv("topoconfig.json")
obs = erEnv.reset()
print("trajectory (len) : ", len(erEnv.trajectory) ,erEnv.trajectory)
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for i in range (len(erEnv.trajectory)):
    print("masks:", erEnv.action_masks())
    print("current_cell:", erEnv.trajectory[i])
    obs, rewards, dones, info = erEnv.step(3)
    print(obs)
    print(rewards)
    print(dones)

trajectory (len) :  11 [2, 14, 16, 23, 35, 37, 42, 38, 28, 21, 17]
App reqlatency:  10 current MEC ID:  mec12 current cell ID:  2
masks: [False, False, False, False, False, False, True, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False]
current_cell: 2
{'space_App': array([[ 1.,  2.,  1.,  4., 14.]]), 'space_MEC': array([[ 1., 29.,  1., 29.,  3.],
       [ 1., 88.,  1., 88.,  3.],
       [ 1., 81.,  1., 81.,  3.],
       [ 1., 27.,  1., 27.,  3.],
       [ 1.,  0.,  1.,  1.,  3.],
       [ 1., 89.,  1., 89.,  3.],
       [ 1., 83.,  1., 83.,  3.],
       [ 1., 19.,  1., 19.,  3.],
       [ 1., 14.,  1., 14.,  3.],
       [ 1., 87.,  1., 87.,  3.],
       [ 1., 10.,  1., 10.,  3.],
       [ 1., 63.,  1., 63.,  3.],
       [ 1., 17.,  1., 17.,  3.],
       [ 1., 63.,  1., 63.,  3.],
       [ 1., 67.,  1., 67.,  3.],
       [ 1., 72.,  1., 72.,  3.],
       [ 2., 49.,  2., 49.,  2.],
       [ 2., 10.,  2., 10.,  2.],
       [ 2.,  1.,  2.